In [ ]:

import configparser
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from binance.client import Client

In [ ]:
CFG_FL_NAME = "../user.cfg"
USER_CFG_SECTION = "binance_user_config"
config = configparser.ConfigParser()
config.read(CFG_FL_NAME)

api_key = config.get(USER_CFG_SECTION, "api_key")
api_secret = config.get(USER_CFG_SECTION, "api_secret_key")
client = Client(api_key, api_secret)

In [ ]:
def get_ticker_price(ticker_symbol: str, days:int ):
    """
    Get ticker price of a specific coin
    """

    target_date = (datetime.now() -timedelta(days = days)).strftime("%d %b %Y %H:%M:%S")
    key = f"{ticker_symbol}"
    end_date = datetime.now() 
    end_date = end_date.strftime("%d %b %Y %H:%M:%S")
    
    coindata = pd.DataFrame(columns = [key])
    
    prices = []
    dates = []
    for result in client.get_historical_klines(
        ticker_symbol, "1m", target_date, end_date, limit=1000
        ):
        date = datetime.utcfromtimestamp(result[0] / 1000).strftime("%d %b %Y %H:%M:%S")
        price = float(result[1])
        dates.append(date)
        prices.append(price)

    coindata[key] = prices
    coindata['date'] = dates

    return(coindata.reindex(columns =['date',key]))

In [ ]:
# Grab coin prices from binance server 
# Add your own obviously


coinIDs = set(line.strip() for line in open('../supported_coin_list'))

cointickers = [coin + 'USDT' for coin in coinIDs]

### Set Period here, this is the number of days of data you want to download for each coin ###
num_days = 1
###

#Create initial df with first coin then fill with all from list

coindata = get_ticker_price(cointickers[0], num_days)

for tick in cointickers[1:]:
    newdata = get_ticker_price(tick, num_days)
    coindata = pd.merge(coindata, newdata)



In [ ]:
## Sanity checker, Are we getting the data? Let's plot it

def plotmovement(x1, y1,ticker):
    '''
    plots the noise of two coins 
    '''
    fig = plt.figure(figsize=(20,8))
    plt.plot( x1, y1,label=ticker)
    plt.legend()

## Too much data so lets just plot a subset.
    
start = 0
end = 200

fig = plt.figure(figsize=(20,8))
coindata.plot('date','BTCUSDT',figsize = (16,8),)

In [ ]:
# print correlation table

coindata.corr()

In [ ]:
## Correlation matrix for raw data. 

corrMatrix = coindata.corr()
fig = plt.figure(figsize=(20,20))
sn.heatmap(corrMatrix, annot=True)
plt.title('Correlation for raw data ')
plt.show()

In [ ]:
### Detrend the data using first difference 
### more info at https://youtu.be/nPYPyh20gGo

detrendeddata = pd.DataFrame()

for column in coindata:
    if column != 'date':
        detrendeddata[column] = coindata[column].diff(1)
        
corrMatrix = detrendeddata.corr()
fig = plt.figure(figsize=(20,20))
sn.heatmap(corrMatrix, annot=True)
plt.title('Correlation for detrended data (diff=1)')
plt.show()

In [ ]:
### Detrend the data using rolling average
radf = pd.DataFrame()

for column in coindata:
    if column != 'date':
        radf[column] = coindata[column]-coindata[column].rolling(window=3).mean()
        
corrMatrix = radf.corr()
fig = plt.figure(figsize=(20,20))
sn.heatmap(corrMatrix, annot=True)
plt.title('Correlation for detrended data (diff=1)')
plt.show()

In [ ]:
### Examine the trend of the rolling average

radf2 = pd.DataFrame()

for column in coindata:
    if column != 'date':
        radf2[column] = coindata[column].rolling(window=3).mean()

corrMatrix = radf2.corr()
fig = plt.figure(figsize=(20,20))
sn.heatmap(corrMatrix, annot=True)
plt.title('Correlation for detrended data (diff=1)')
plt.show()